In [ ]:
# Import pieces from codeflare-sdk
from codeflare_sdk.cluster.cluster import Cluster, ClusterConfiguration
from codeflare_sdk.cluster.auth import TokenAuthentication
from codeflare_sdk.job.jobs import DDPJobDefinition

In [ ]:
# Create authentication object for oc user permissions
auth = TokenAuthentication(
    token = "sha256~2CRnJ7Tef3nTsAlWowaedwCUcc4lLswrNtpoLnON6F4",
    server = "https://api.mini2.mydomain.com:6443",
    skip_tls=True
)
auth.login()

Here, we want to define our cluster by specifying the resources we require for our batch workload. Below, we define our cluster object (which generates a corresponding AppWrapper).

In [ ]:
# Create our cluster and submit appwrapper (reduce specs as desired)
cluster = Cluster(ClusterConfiguration(
    name='mnisttest',
    namespace='batch-mnist',
    image="quay.io/thinkahead/base:ray2.1.0-py38-gpu-pytorch1.12.0cu117-20230419-1",
    min_worker=2,
    max_worker=3,
    min_cpus=8,
    max_cpus=8,
    min_memory=16,
    max_memory=16,
    gpu=1,
    instascale=False # Can be set to false if scaling not needed
))

Next, we want to bring our cluster up, so we call the `up()` function below to submit our cluster AppWrapper yaml onto the MCAD queue, and begin the process of obtaining our resource cluster.

In [ ]:
# Bring up the cluster
cluster.up()

Now, we want to check on the status of our resource cluster, and wait until it is finally ready for use.

In [ ]:
cluster.status()

In [ ]:
cluster.wait_ready()

In [ ]:
cluster.status()

Let's quickly verify that the specs of the cluster are as expected.

In [ ]:
cluster.details()

Now that our resource cluster is ready, we can directly submit our batch job (model training on three workers with 1 gpu each) to the cluster via torchx.

In [ ]:
jobdef = DDPJobDefinition(
    name="mnisttest",
    script="mnist_fashion.py",
    #scheduler_args={"requirements": "requirements.txt"}
)
job = jobdef.submit(cluster)

Now we can go ahead and look at the status and logs of our batch job.

In [ ]:
job.status()

In [ ]:
print(job.logs())

Finally, we bring our resource cluster down and release/terminate the associated resources, bringing everything back to the way it was before our cluster was brought up.

In [ ]:
cluster.down()

In [ ]:
auth.logout()

# Fetch the Fashion-MNIST dataset

In [ ]:
!pip install scikit-learn matplotlib

In [ ]:
from sklearn.datasets import fetch_openml
import numpy as np
import json
import requests
X, y = fetch_openml('Fashion-MNIST', return_X_y=True, parser='auto')

In [ ]:
X.shape

# Draw few images

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# build a dictionary for easy access to object classes
objects = {0: 'T-shirt/top',
           1: 'Trouser',
           2: 'Pullover',
           3: 'Dress',
           4: 'Coat',
           5: 'Sandal',
           6: 'Shirt',
           7: 'Sneaker',
           8: 'Bag',
           9: 'Ankle boot'}
# let's have a quick look of those images
f, axes = plt.subplots(2, 4)
for row in axes:
    for axe in row:
        index = np.random.randint(10000)
        img = np.array(X.iloc[index, 0:]).reshape((28, 28))
        obj = X.iloc[index, 0]
        axe.imshow(img, cmap='gray')
        axe.set_title(objects[obj])

In [ ]:
from matplotlib import pyplot
plotX=X.loc[0:8].values.reshape(9,28,28)
# plot first few images
for i in range(9):
    pyplot.subplot(330 + 1 + i)
    pyplot.imshow(plotX[i], cmap=pyplot.get_cmap('gray'))
# show the figure
pyplot.show()

# Load the onnx model

In [ ]:
#model_name='mnist1' # torch.onnx.export
#model_name='mnist2' # Pytorch Lightning model.to_onnx
model_name='mnist3' # torch.onnx.export with batch
#model_name='mnist4' # Pytorch Lightning model.to_onnx with batch

model_file_name=model_name+".onnx"

In [ ]:
import onnx
import onnxruntime
import torch
import numpy as np
session = onnxruntime.InferenceSession(model_file_name, None, providers=['CPUExecutionProvider'])
input_name = session.get_inputs()[0].name
print("input name", input_name)
input_shape = session.get_inputs()[0].shape
print("input shape", input_shape)
input_type = session.get_inputs()[0].type
print("input type", input_type)
print([i.name for i in session.get_outputs()])
output_name = session.get_outputs()[0].name
print("output name", output_name)
output_shape = session.get_outputs()[0].shape
print("output shape", output_shape)
output_type = session.get_outputs()[0].type
print("output type", output_type)

# Batch request shape (batch_size,1,28,28)

In [ ]:
batch=X[0:5]
actual=y[0:5]
arr=np.array(batch)
result = session.run([i.name for i in session.get_outputs()], {input_name:(arr.reshape(len(batch),1,28,28).astype(np.float32))})

In [ ]:
print(np.argmax(result[0],axis=1),actual.values)

# Single request shape (1,1,28,28) - Loop through 10000 samples

In [ ]:
%%time
count=0
#for i in X.index:
for i in range(10000):
    arr=np.array(X.iloc[i])
    result = session.run([i.name for i in session.get_outputs()], {input_name:(arr.reshape(1,*input_shape[1:]).astype(np.float32))})
    expected=int(y.iloc[i])
    actual=np.argmax(result)
    if actual!=expected: count+=1
    #print("Expected",expected,"Actual",actual==expected)
print('Accuracy:',(1-count/len(X)))

In [ ]:
# classes of fashion mnist dataset
classes = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle Boot']
# plotting the results
fig = plt.figure(figsize=(25,4))
for imagenum in range(20):
    arr=np.array(X.iloc[imagenum])
    result = session.run([i.name for i in session.get_outputs()], {input_name:(arr.reshape(1,*input_shape[1:]).astype(np.float32))})
    expected=int(y.iloc[imagenum])
    actual=np.argmax(result)
    ax = fig.add_subplot(2, 10, imagenum+1, xticks=[], yticks=[])
    ax.imshow(np.array(X.iloc[imagenum, 0:]).reshape((28, 28)))
    ax.set_title("{} ({})".format(classes[expected], classes[actual]),
                 color=("green" if expected==actual else "red"))

# Copy onnx model to S3 bucket

In [ ]:
import os
import boto3
from boto3 import session
key_id = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
session = boto3.session.Session(aws_access_key_id=key_id, aws_secret_access_key=secret_key)
s3_client = boto3.client('s3', aws_access_key_id=key_id, aws_secret_access_key=secret_key,endpoint_url=endpoint_url,verify=False)
buckets=s3_client.list_buckets()
for bucket in buckets['Buckets']: print(bucket['Name'])
s3_client.upload_file(model_file_name, bucket['Name'],model_file_name)
[item.get("Key") for item in s3_client.list_objects_v2(Bucket=bucket['Name']).get("Contents")]

In [ ]:
import os
import boto3
from boto3 import session
key_id = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
session = boto3.session.Session(aws_access_key_id=key_id, aws_secret_access_key=secret_key)
s3_client = boto3.client('s3', aws_access_key_id=key_id, aws_secret_access_key=secret_key,endpoint_url=endpoint_url,verify=False)
buckets=s3_client.list_buckets()
s3_client.upload_file(model_name+"/"+model_name+".bin", bucket['Name'],"mymodel/"+model_name+".bin")
s3_client.upload_file(model_name+"/"+model_name+".xml", bucket['Name'],"mymodel/"+model_name+".xml")
[item.get("Key") for item in s3_client.list_objects_v2(Bucket=bucket['Name']).get("Contents")]

Deploy the model in your Data Science project using RHODS UI under "Models and model servers"

# Submit HTTP REST request to the ModelMesh for single sample

In [ ]:
model_name="mnist1"

import requests
import json
URL='http://modelmesh-serving.huggingface.svc.cluster.local:8008/v2/models/'+model_name+'/infer' # underscore characters are removed
headers = {}
payload = {
        "inputs": [{ "name": "input_0", "shape": (1,1,28,28), "datatype": "FP32", "data": X.iloc[0].values.tolist()}]
    }
headers = {"content-type": "application/json"}
res = requests.post(URL, json=payload, headers=headers)
print(res)
print(res.text)

In [ ]:
print("Expected",y.iloc[0],"Actual",np.argmax(res.json()['outputs'][0]['data']))

# Submit HTTP REST request to the ModelMesh for a batch of samples

In [ ]:
model_name="mymodel"

import requests
import json
URL='http://modelmesh-serving.huggingface.svc.cluster.local:8008/v2/models/'+model_name+'/infer' # underscore characters are removed
headers = {}
payload = {
        "inputs": [{ "name": "input_0", "shape": (5,1,28,28), "datatype": "FP32", "data": X.loc[0:4].values.flatten().tolist()}]
    }
#print(payload)
headers = {"content-type": "application/json"}
res = requests.post(URL, json=payload, headers=headers)
print(res)
print(res.text)

In [ ]:
actual=y[0:5].values.tolist()
expected=np.argmax(np.array(res.json()['outputs'][0]['data']).reshape(res.json()['outputs'][0]['shape']),axis=1)
print(actual,expected)

# Submit gRPC request to the ModelMesh for single sample

In [ ]:
!pip install grpcio grpcio-tools==1.46.0

In [ ]:
#!wget https://raw.githubusercontent.com/kserve/kserve/master/docs/predict-api/v2/grpc_predict_v2.proto
!wget https://raw.githubusercontent.com/kserve/modelmesh-serving/main/fvt/proto/kfs_inference_v2.proto
!python3 -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. ./kfs_inference_v2.proto

In [ ]:
model_name="mnist1" # mnist2, mnist3, mnist4
payload = { "model_name": model_name,
            "inputs": [{ "name": "input_0", "shape": (1,1,28,28), "datatype": "FP32", "contents": {"fp32_contents":X.iloc[0].values.tolist()}}]
    }

In [ ]:
import grpc
import kfs_inference_v2_pb2, kfs_inference_v2_pb2_grpc
grpc_url="modelmesh-serving.huggingface.svc.cluster.local:8033"
request=kfs_inference_v2_pb2.ModelInferRequest(model_name=model_name,inputs=payload["inputs"])
grpc_channel = grpc.insecure_channel(grpc_url)
grpc_stub = kfs_inference_v2_pb2_grpc.GRPCInferenceServiceStub(grpc_channel)
response = grpc_stub.ModelInfer(request)

In [ ]:
print(type(response.outputs),type(response.raw_output_contents))
from google.protobuf.json_format import MessageToDict
d = MessageToDict(response.outputs[0])
print(d)
binary_data=bytes([x for x in response.raw_output_contents[0]])

In [ ]:
import struct
import base64
FLOAT = 'f'
fmt = '<' + FLOAT * (len(binary_data) // struct.calcsize(FLOAT))
print("Expected",y.iloc[0],"Actual",np.argmax(np.array(struct.unpack(fmt, binary_data))))

# Submit gRPC request to the ModelMesh for batch of samples

In [ ]:
model_name="mnist3"  # mnist4
payload = { "model_name": model_name,
            "inputs": [{ "name": "input_0", "shape": (5,1,28,28), "datatype": "FP32", "contents": {"fp32_contents":X.loc[0:4].values.flatten().tolist()}}]
    }

In [ ]:
import grpc
import kfs_inference_v2_pb2, kfs_inference_v2_pb2_grpc
grpc_url="modelmesh-serving.huggingface.svc.cluster.local:8033"
request=kfs_inference_v2_pb2.ModelInferRequest(model_name=model_name,inputs=payload["inputs"])
grpc_channel = grpc.insecure_channel(grpc_url)
grpc_stub = kfs_inference_v2_pb2_grpc.GRPCInferenceServiceStub(grpc_channel)
response = grpc_stub.ModelInfer(request)

In [ ]:
print(type(response.outputs),type(response.raw_output_contents))
from google.protobuf.json_format import MessageToDict
d = MessageToDict(response.outputs[0])
print(d)
binary_data=bytes([x for x in response.raw_output_contents[0]])

In [ ]:
import struct
import base64
FLOAT = 'f'
fmt = '<' + FLOAT * (len(binary_data) // struct.calcsize(FLOAT))
numbers = [str(n) for n in np.argmax(np.array(struct.unpack(fmt, binary_data)).reshape(*[int(shapeval) for shapeval in d['shape']]),axis=1)]
print("Expected",y[0:5].values.tolist(),"Actual",numbers)